see: https://www.overleaf.com/read/knktnhbycdzg#210acd

In [1]:
import sys
sys.path.insert(0, '..')

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

L = np.pi
N = 256

theta = np.linspace(-np.pi, np.pi, N)

def plot(A, B, C, w0, w1):
    V = lambda theta: A + B * np.cos(theta) + C * np.cos(theta)**2
    
    V_non_negative = np.abs(V(theta)) #np.maximum(V(theta), 0)
    delta_W = lambda theta: np.sqrt(V_non_negative) * np.random.randn(len(theta))

    W = lambda delta_theta: w0 + w1 * np.cos(delta_theta) + delta_W(delta_theta)

    fig, axs = plt.subplots(1, 3, figsize=(24, 6)) 

    axs[0].plot(theta, V(theta), 'r-', markersize=2)
    axs[0].axhline(0, color='gray', linestyle='--')
    axs[0].set_xlabel('Theta')
    axs[0].set_ylabel('V(theta)')
    axs[0].set_title('V(theta)')
    axs[0].grid(True)
    axs[0].legend(['V'])

    axs[1].plot(theta, delta_W(theta), 'b-', markersize=2)
    axs[1].set_xlabel('Theta')
    axs[1].set_ylabel('delta_W(theta)')
    axs[1].set_title('delta_W(theta)')
    axs[1].grid(True)
    axs[1].legend(['delta_W'])

    axs[2].plot(theta, W(theta), 'g-', markersize=2)
    axs[2].set_xlabel('Theta')
    axs[2].set_ylabel('W(theta)')
    axs[2].set_title('W(theta)')
    axs[2].grid(True)
    axs[2].legend(['W'])

    plt.show()

A_slider = FloatSlider(min=-5, max=5, step=1, value=0.5, description='A')
B_slider = FloatSlider(min=-5, max=5, step=1, value=0, description='B')
C_slider = FloatSlider(min=-5, max=5, step=1, value=0, description='C')
w0_slider = FloatSlider(min=-30, max=30, step=1, value=-10, description='w0')
w1_slider = FloatSlider(min=-30, max=30, step=1, value=10, description='w1')

interact(plot, A=A_slider, B=B_slider, C=C_slider, w0=w0_slider, w1=w1_slider)


interactive(children=(FloatSlider(value=0.5, description='A', max=5.0, min=-5.0, step=1.0), FloatSlider(value=…

<function __main__.plot(A, B, C, w0, w1)>

In [11]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive, IntSlider, FloatSlider
import ipywidgets as widgets

default_N = 64
noise = np.random.randn(default_N)
previous_N = default_N  # Track the last value of N

V = lambda theta, A, B, C: A + B * np.cos(theta) + C * np.cos(theta)**2
delta_W = lambda theta, A, B, C, noise: np.sqrt(V(theta, A, B, C)) * noise

def fourier_series(theta_values, J, delta_W_values):
    fourier_reconstruction = np.zeros_like(theta_values)
    contributions = np.zeros_like(theta_values)
    amplitude = 0  
    phase = 0      
    for j in range(1, J+1):
        alpha_j, beta_j = fourier_coefficients(j, theta_values, delta_W_values)
        contribution = alpha_j * np.cos(j * theta_values) + beta_j * np.sin(j * theta_values)
        fourier_reconstruction += contribution
        if j == J:
            contributions = contribution
            amplitude = np.sqrt(alpha_j**2 + beta_j**2)
            phase = np.arctan2(beta_j, alpha_j)
    return fourier_reconstruction, contributions, amplitude, phase

def interactive_plot(J, A, B, C, N, w0, w1):
    global noise, previous_N
    theta_values = np.linspace(-np.pi, np.pi, N)
    
    if N != previous_N:  # Check if N has changed
        noise = np.random.randn(N)
        previous_N = N  # Update the previous_N to current N

    delta_W_values = delta_W(theta_values, A, B, C, noise)
    reconstructed_signal, last_mode_contribution, amplitude, phase = fourier_series(theta_values, J, delta_W_values)

    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(24, 6))
    
    ax1.plot(theta_values, delta_W_values, label='Original $\Delta W$', alpha=0.5)
    ax1.plot(theta_values, reconstructed_signal, label=f'Fourier Series up to {J} modes', linestyle='--')
    ax1.set_title("Fourier Series Approximation of $\Delta W$")
    ax1.set_xlabel("$\\theta$")
    ax1.set_ylabel("$\Delta W$")
    ax1.legend()
    
    ax2.plot(theta_values, last_mode_contribution, color='red',
             label=f'Contribution of Mode {J}\nAmplitude: {amplitude:.3f}\nPhase: {phase:.3f} rad')
    ax2.set_title(f"Contribution of Mode {J}")
    ax2.set_xlabel("$\\theta$")
    ax2.set_ylabel("Contribution to $\Delta W$")
    ax2.legend()

    kernel = w0 + w1 * np.cos(theta_values) + reconstructed_signal
    amplitude = np.max(kernel) - np.min(kernel)
    
    max_index = np.argmax(kernel)  
    phase = theta_values[max_index] 

    ax3.plot(theta_values, kernel, color='blue', label=f'Kernel $W(\\delta \\theta)$\nAmplitude: {amplitude:.2f}, Phase: {phase} rad')
    ax3.set_title(f"Kernel with Fourier Series truncated at {J}")
    ax3.set_xlabel("$\\delta \\theta$")
    ax3.set_ylabel("$W(\\delta \\theta)$")
    ax3.legend()

interactive_plot = interactive(
    interactive_plot,
    J=IntSlider(min=0, max=128, step=1, value=0, description='J: Number of Modes'),
    A=FloatSlider(min=0, max=9, step=1, value=1, description='A'),
    B=FloatSlider(min=0, max=9, step=1, value=0, description='B'),
    C=FloatSlider(min=0, max=9, step=1, value=0, description='C'),
    N=IntSlider(min=16, max=1025, step=16, value=default_N, description='N: Number of Points'),
    w0=FloatSlider(min=-20, max=10, step=1, value=-10, description='w0'),
    w1=FloatSlider(min=-20, max=20, step=1, value=10, description='w1')
)
display(interactive_plot)

interactive(children=(IntSlider(value=0, description='J: Number of Modes', max=128), FloatSlider(value=1.0, de…

In [4]:
from common_utils import Ring, derivative_nonlinearity, r_02
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

np.random.seed(42)

L = np.pi
N = 255
T = {'t_span': (0, 5000), 't_steps': 5000}

w0 = -10
I_0 = 0.9

A = 0.05
B = 0.0 
C = 0.0

M = 60  
s = 0.1  
kappa = s*M

R = np.sqrt((2*np.pi/N)*(A+(C/2))) * (np.sqrt(s*M)/kappa)

r_0 = r_02(w0, I_0)
critical_w1 = 2 / derivative_nonlinearity(w0 * r_0 + I_0) -  R
print(f'Critical W_1: {critical_w1:.4f}')
print(f'Previous Critical W_1 at Delta: {R:.4f}')

print(f'Solution r0={r_0:.4f}')

def update_plots(epsilon, delta):
    theta = np.linspace(-np.pi, np.pi, N)
    w1 = critical_w1 + delta
    perturbation = lambda theta: r_0 + epsilon * np.cos(theta)

    V = lambda theta: A + B * np.cos(theta) + C * np.cos(theta)**2
    
    delta_W = lambda theta: np.sqrt(V(theta)) * np.random.randn()
    
    W = lambda delta_theta: w0 + w1 * np.cos(delta_theta) + delta_W(delta_theta)

    ring = Ring(L, T, N, W, I_0, perturbation)

    fig, axs = plt.subplots(1, 3, figsize=(24, 6))  

    ring.plot_timetrace(ax=axs[0], phase=0)
    ring.plot_state(ax=axs[1], timestep=-1)
    ring.plot_dynamics(ax=axs[2])
    
    plt.tight_layout()
    plt.show()

epsilon_slider = FloatSlider(value=0.005, min=0.0, max=0.1, step=0.005, description='Epsilon:', readout_format='.3f')
delta_slider = FloatSlider(value=0.0, min=-0.5, max=0.5, step=0.05, description='Delta:', readout_format='.2f')

interact(update_plots, epsilon=epsilon_slider, delta=delta_slider)

Critical W_1: 3.9205
Previous Critical W_1 at Delta: 0.0143
Solution r0=0.0646


interactive(children=(FloatSlider(value=0.005, description='Epsilon:', max=0.1, readout_format='.3f', step=0.0…

<function __main__.update_plots(epsilon, delta)>

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider
from common_utils import derivative_nonlinearity, r_02

w0 = -30
I_0 = 2

r_0 = r_02(w0, I_0)
phi_0_prime = derivative_nonlinearity(w0 * r_0 + I_0)

def plot_graphs(A, B, C, N, W1):
    R1 = np.sqrt(2 * np.pi / N * (A + C / 2))
    psi_1 = np.linspace(-np.pi, np.pi, 400)  # psi_1 values from -pi to pi
    
    # Compute delta
    tan_delta = R1 * np.sin(psi_1) / (B + R1 * np.cos(psi_1))
    delta = np.arctan(tan_delta)  # Compute delta using arctan

    # Compute R
    R = np.sqrt(W1**2 + 2 * W1 * R1 * np.cos(psi_1) + R1**2)
    
    # Compute critical W1 using the given formula
    W1_cr = 2/phi_0_prime - R1 #-R1 * np.cos(psi_1) + np.sqrt((2 / phi_0_prime)**2 - R1**2 * np.sin(psi_1)**2)

    # Setup plots
    fig, ax = plt.subplots(1, 3, figsize=(18, 5))
    
    # Plot delta as a function of psi_1
    ax[0].plot(psi_1, delta, label=r'$\delta(\psi_1)$')
    ax[0].set_title('Plot of $\delta$ as a Function of $\psi_1$')
    ax[0].set_xlabel('$\psi_1$ (radians)')
    ax[0].set_ylabel('$\delta$ (radians)')
    ax[0].grid(True)
    ax[0].legend()

    # Plot R as a function of psi_1
    ax[1].plot(psi_1, R, label=r'$R(\psi_1)$', color='blue')
    ax[1].set_title('Plot of $R$ as a Function of $\psi_1$')
    ax[1].set_xlabel('$\psi_1$ (radians)')
    ax[1].set_ylabel('$R$')
    ax[1].grid(True)
    ax[1].legend()

    # Plot critical W1 as a function of psi_1
    ax[2].plot(psi_1, W1_cr, label=r'$W_{1}^{cr}(\psi_1)$', color='red')
    ax[2].axhline(y=2/phi_0_prime, color='red', linestyle='--', label='2/phi_0\'')
    ax[2].set_title('Critical $W_{1}^{cr}$ as a Function of $\psi_1$')
    ax[2].set_xlabel('$\psi_1$ (radians)')
    ax[2].set_ylabel('$W_{1}^{cr}$')
    ax[2].grid(True)
    ax[2].legend()

    plt.show()

# Create interactive sliders for A, B, C, N, and W1
interact(plot_graphs,
         A=FloatSlider(value=1, min=0, max=10, step=0.1, description='A:'),
         B=FloatSlider(value=1, min=0, max=10, step=0.1, description='B:'),
         C=FloatSlider(value=0, min=0, max=10, step=0.1, description='C:'),
         N=FloatSlider(value=64, min=10, max=100, step=1, description='N:'),
         W1=FloatSlider(value=1, min=0, max=5, step=0.1, description='W1:')
        );


interactive(children=(FloatSlider(value=1.0, description='A:', max=10.0), FloatSlider(value=1.0, description='…

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider
from common_utils import Ring, fourier_coefficients, r_02, derivative_nonlinearity

w0 = -10
I_0 = 0.9
w1 = 6
A = 1.0
B = 0.0
C = 0.0
r_0 = r_02(w0, I_0)

def fourier_kernel(theta_values, J, A, B, C, w0, w1):
    V = A + B * np.cos(theta_values) + C * np.cos(theta_values)**2
    delta_W_values = np.sqrt(V) * np.random.randn(len(theta_values))
    
    fourier_reconstruction = np.zeros_like(theta_values)
    
    for j in range(1, J + 1):
        alpha_j, beta_j = fourier_coefficients(j, theta_values, delta_W_values)
        fourier_reconstruction += alpha_j * np.cos(j * theta_values) + beta_j * np.sin(j * theta_values)

    W = lambda delta_theta: w0 + w1 * np.cos(delta_theta) + np.interp(delta_theta, theta_values, fourier_reconstruction)
    return W

L=np.pi
N=256
T={'t_span': (0, 5000), 't_steps': 5000}

def update_plots(J):
    theta_values = np.linspace(-np.pi, np.pi, N)
    
    perturbation = lambda theta: r_0 + 0.005 * np.cos(theta)
    W = fourier_kernel(theta_values, J, A, B, C, w0, w1)
    
    ring = Ring(L, T, N, W, I_0, perturbation)
    
    fig, axs = plt.subplots(1, 3, figsize=(24, 6))
    
    axs[0].plot(theta_values, W(theta_values), label=f'Connectivity Kernel up to mode {J}')
    axs[0].set_title("Kernel Function")
    axs[0].set_xlabel("$\\delta \\theta$")
    axs[0].set_ylabel("$W(\\delta \\theta)$")
    axs[0].legend()

    ring.plot_state(ax=axs[1], timestep=-1)
    ring.plot_dynamics(ax=axs[2])
    
    plt.tight_layout()
    plt.show()

J_slider = IntSlider(min=1, max=N, step=1, value=1, description='J: Number of Modes')

interact(update_plots, J=J_slider)

interactive(children=(IntSlider(value=1, description='J: Number of Modes', max=256, min=1), Output()), _dom_cl…

<function __main__.update_plots(J)>

In [19]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact

# Assuming these functions return a scalar value
def r_02(w0, I_0):
    return 1.0  # Placeholder function to compute r_0

def derivative_nonlinearity(input):
    return 1.0  # Placeholder derivative function of the nonlinearity

w0 = -10
I_0 = 0.9
w1 = 6
r_0 = r_02(w0, I_0)

phi0_prime = derivative_nonlinearity(r_0 * w0 + I_0)

def plot_W1_cr(psi1, R1):
    theta = np.linspace(-1, 1, 1000)
    W1_cr = 2 / phi0_prime - R1 * np.cos(theta + psi1) / np.cos(theta)
    
    plt.figure(figsize=(10, 5))
    plt.plot(theta, W1_cr, label=f'$\psi_1 = {psi1:.2f}, R_1 = {R1:.2f}$')
    plt.axhline(y=2/phi0_prime, color='red', linestyle='--', label=f'y = {2/phi0_prime}')  # Horizontal line at y=3
    plt.title("Plot of $W_1^{cr}$ as a Function of $\\theta$")
    plt.xlabel("$\\theta$")
    plt.ylabel("$W_1^{cr}$")
    plt.legend()
    plt.grid(True)
    plt.show()

interact(plot_W1_cr, psi1=(-np.pi, np.pi, 0.1), R1=(0.0, 1.0, 0.1))

interactive(children=(FloatSlider(value=-0.04159265358979303, description='psi1', max=3.141592653589793, min=-…

<function __main__.plot_W1_cr(psi1, R1)>